In [ ]:
import os
import cv2
import sys
import time
import random
import pickle
import skimage
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.cluster import DBSCAN, KMeans

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LambdaCallback, CSVLogger

from general_utils import encode_mask_to_rgb, get_normalized_image
from data_loader import DataLoader
from model_pipeline import ModelPipeline
from predictions_pipeline import PredictionsPipeline

In [ ]:
embeddings_df = pd.read_pickle('data//embeddings_df.pkl')
data_dir = 'data//dataset_for_m1_model_combined//'
rgb_df_fp = 'data//rgb_calculations.pkl'
rgb_df = pd.read_pickle(rgb_df_fp)
model_name = 'saved_models//supervised_unet.h5'
#vgg_weights_path = "VGG16NOTOP.h5"
im_height = 256
im_width = 768

data_loader = DataLoader(path=data_dir, im_height=im_height, im_width=im_width)
model_pipeline = ModelPipeline(batch_size=8)

In [ ]:
X, Y, X_ids = data_loader.get_only_labeled_data(path=data_dir, masks_folder="masks_all")

In [ ]:
train_dataset, valid_dataset, params = model_pipeline.get_images_and_masks_datasets(X, Y, channel_normalize=False)

In [ ]:
def get_unet(kernel = 3, pool_size = (2, 2), input_height=256, input_width=768, channel=3, n_classes=5):

    img_input = Input(shape=(input_height, input_width, channel))

    ###################### Encoder 1 ############################
    c1 = Convolution2D(64, (kernel, kernel), padding="same", activation='selu')(img_input)
    c2 = Convolution2D(64, (kernel, kernel), padding="same", activation='selu')(c1)
    p1 = MaxPooling2D(pool_size)(c1)

    ###################### Encoder 2 ############################
    c2 = Convolution2D(128, (kernel, kernel), padding="same", activation='selu')(p1)
    c2 = Convolution2D(128, (kernel, kernel), padding="same", activation='selu')(c2)
    p2 = MaxPooling2D(pool_size)(c2)

    ###################### Encoder 3 ############################
    c3 = Convolution2D(256, (kernel, kernel), padding="same", activation='selu')(p2)
    c3 = Convolution2D(256, (kernel, kernel), padding="same", activation='selu')(c3)
    p3 = MaxPooling2D(pool_size)(c3)
    p3 = Dropout(0.2)(p3)

    ###################### Encoder 4 ############################
    c4 = Convolution2D(512, (kernel, kernel), padding="same", activation='selu')(p3)
    c4 = Convolution2D(512, (kernel, kernel), padding="same", activation='selu')(c4)
    p4 = MaxPooling2D(pool_size)(c4)
    p4 = Dropout(0.2)(p4)

    ###################### BOTTLENECK ###########################
    d1 = Convolution2D(512, (kernel, kernel), padding="same", activation='selu')(p4)
    d1 = Convolution2D(512, (kernel, kernel), padding="same", activation='selu')(d1)
    d1 = Dropout(0.5)(d1)

    ###################### Decoder 1 ############################
    u1 = concatenate([UpSampling2D(2)(d1), c4])
    c5 = Convolution2D(512, (kernel, kernel), padding="same")(u1)
    c5 = BatchNormalization()(c5)
    c5 = Activation('selu')(c5)
    c5 = Convolution2D(256, (kernel, kernel), padding="same")(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('selu')(c5)
    c5 = Dropout(0.2)(c5)

    ###################### Decoder 2 ############################
    u2 = concatenate([UpSampling2D(2)(c5), c3])
    c6 = Convolution2D(256, (kernel, kernel), padding="same")(u2)
    c6 = BatchNormalization()(c6)
    c6 = Activation('selu')(c6)
    c6 = Convolution2D(128, (kernel, kernel), padding="same")(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('selu')(c6)
    c6 = Dropout(0.2)(c6)

    ###################### Decoder 3 ############################
    u3 = concatenate([UpSampling2D(2)(c6), c2])
    c7 = Convolution2D(128, (kernel, kernel), padding="same")(u3)
    c7 = BatchNormalization()(c7)
    c7 = Activation('selu')(c7)
    c7 = Convolution2D(64, (kernel, kernel), padding="same")(c7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('selu')(c7)

    ###################### Decoder 4 ############################
    u4 = concatenate([UpSampling2D(2)(c7), c1])
    c8 = Convolution2D(64, (kernel, kernel), padding="same")(u4)
    c8 = BatchNormalization()(c8)
    c8 = Activation('selu')(c8)
    c8 = Convolution2D(64, (kernel, kernel), padding="same")(c8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('selu')(c8)

    c8 = Convolution2D(n_classes, (kernel, kernel), padding="same")(c8)

    # Output layer must be manually cast to float32 when using mixed precision
    softmax_outputs = tf.keras.layers.Activation("softmax", name='softmax_predictions', dtype="float32")(c8)

    model = tf.keras.Model(img_input, softmax_outputs)
    return model

In [ ]:
model = get_unet(input_height=im_height, input_width=im_width)

In [ ]:
img_num = 120 #  # try different numbers to find a good example to plot during training
image = X[img_num]
#sample_image = get_normalized_image(image, rgb_df)
sample_image = image.astype(np.float32) / 255
sample_mask = np.argmax(Y[img_num], axis=-1)
sample_mask = encode_mask_to_rgb(sample_mask)

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 5))
    title = ["Image Input", "Predicted Mask", "True Mask"]
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

def show_predictions(epoch, logs=None):

    prediction = model.predict(sample_image[tf.newaxis, ...])
    prediction = np.reshape(prediction, (prediction.shape[1], prediction.shape[2], prediction.shape[3]))
    prediction = np.argmax(prediction, axis=-1)
    prediction = encode_mask_to_rgb(prediction)
    display([image.astype(np.uint8), prediction, sample_mask])

show_predictions(epoch=0)

In [ ]:
# try: longer patience for ReduceLROnPlateau
callbacks = [
    LambdaCallback(on_epoch_end=show_predictions),
    EarlyStopping(monitor='val_dice_coef', mode = 'max', patience=15, verbose=2), 
    ReduceLROnPlateau(monitor='val_dice_coef', factor=0.1, patience=5, min_lr=1e-10, mode = 'max', verbose=2),
    ModelCheckpoint(model_name, monitor='val_dice_coef', verbose=2, mode='max', save_best_only=True, save_weights_only=True)
]

In [ ]:
model.load_weights(model_name)

In [ ]:
model = model_pipeline.compile_and_train_supervised(model, train_dataset, valid_dataset, params, callbacks)

In [ ]:
pred_pipeline = PredictionsPipeline(path=data_dir, model=model, rgb_df_fp=rgb_df_fp, im_height=im_height, im_width=im_width)

In [ ]:
y_uncertainty, y_defect_type = pred_pipeline.export_predictions_and_heatmaps(normalize=False)

In [ ]:
embeddings_df["uncertainty_label"] = y_uncertainty
embeddings_df["defect_label"] = y_defect_type

In [ ]:
# embeddings_df.to_pickle("data//embeddings_df.pkl")

In [ ]:
def create_tsne_graph(df, defect_color_scale=False, discrete_colors=True):
    """Creates tsne graph as a plotly.go figure
    arguments
        df: pandas dataframe with columns for image ids, labels, as well as x and y coordinates of the t-sne embeddings for each datapoint.
        defect_color_scale: when True, colors are from a pre-defined color map 
        discrete_colors: uses a plotly discrete colorscale to show colors for any number of classes
    """
    
    df.to_pickle("data//temp_df.pkl")
    
    if defect_color_scale:
        color_scale = ['rgb(0,0,0)', 'rgb(0,0,255)', 'rgb(255,0,0)', 'rgb(0,255,255)', 'rgb(255,255,0)']
        label_df = df['defect_label']
    elif discrete_colors:
        color_scale = px.colors.qualitative.Alphabet
        label_df = df['cluster_label']
    else:
        color_scale = "thermal"
        label_df = df['uncertainty_label']
        
    fig = go.Figure(
        data=go.Scatter(
            x=df['x'],
            y=df['y'],
            mode='markers',
            marker_size = 8,
            marker_color=label_df,
            marker_colorscale = color_scale,
            marker_opacity = 0.7,
            text=df['Img_ID'],
            customdata = label_df,
            hoverinfo = 'text',
            showlegend=False,
        )
    ) 
    
    fig.update_layout(
        title=dict(
                text="t-SNE Embeddings", 
                x=0.5, 
                font=dict(size=32, family="Calibri Light, HelveticaNeue")
        ),
        legend=dict(orientation='v', x=1, y=0, bordercolor='Grey', borderwidth=1),
        #height=500,
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        paper_bgcolor='rgb(255,255,255)', 
        plot_bgcolor='rgb(255,255,255)'
    )
    
    return fig

In [ ]:
fig = create_tsne_graph(embeddings_df, defect_color_scale=False, discrete_colors=False)
fig.show()

In [ ]:
fig = create_tsne_graph(embeddings_df, defect_color_scale=True)
fig.show()